In [ ]:
from flask import Flask, request, jsonify, send_file
import os
from kitchen_generation import run_kitchen_generation

app = Flask(__name__)

# Allow CORS for both localhost (dev) and deployed UI
@app.after_request
def add_cors_headers(response):
    origin = request.headers.get("Origin")
    allowed_origins = [
        "http://localhost:8080",
        "https://d28c-94-249-51-113.ngrok-free.app"
        "http://localhost:8080" # Your UI domain
    ]
    if origin in allowed_origins:
        response.headers["Access-Control-Allow-Origin"] = origin

    response.headers["Access-Control-Allow-Methods"] = "GET, POST, OPTIONS"
    response.headers["Access-Control-Allow-Headers"] = "Content-Type, Authorization"
    return response

@app.route("/generate_kitchen", methods=["POST"])
def generate_kitchen():
    try:
        print("📥 Received request!")
        data = request.get_json()
        transcript = data.get("transcript", "").strip()

        if not transcript:
            return jsonify({"error": "Transcript is missing"}), 400

        print(f"🗣️ Transcript: {transcript}")
        image_path = "/workspace/stable-diffusion-webui/images/kitchen.jpg"

        if not os.path.exists(image_path):
            return jsonify({"error": f"Static image not found at {image_path}"}), 400

        # Run generation
        final_path = run_kitchen_generation(transcript, image_path)

        # Return local image path (used later in frontend)
        return jsonify({ "image_url": final_path })

    except Exception as e:
        print(f"❌ Error: {e}")
        return jsonify({"error": str(e)}), 500

@app.route("/generated-image")
def send_generated_image():
    from flask import request, send_file, jsonify
    import os

    base_dir = "/workspace/stable-diffusion-webui/outputs"
    relative_path = request.args.get("path", "").replace("..", "").strip()
    full_path = os.path.join(base_dir, relative_path)

    if not os.path.exists(full_path):
        return jsonify({"error": "Image not found"}), 404

    # Just send the image file
    return send_file(full_path, mimetype="image/png")

    return send_file(full_path, mimetype="image/png")




if __name__ == "__main__":
    app.run(host="0.0.0.0", port=4000)